<a href="https://colab.research.google.com/github/hinsley/colabs/blob/master/Training_Parameterization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Motivation

A coach has a set of tools he uses to develop a training plan and continually tailor it to the ever-evolving needs of his client. The tools among coaches across different resistance training disciplines tend to overlap almost completely. If it is possible to construct an expressive and comprehensive unified parameterization of the concepts these tools deal with, then a powerful analytical model of training becomes available to us.

# Metrics

- `AI`: Absolute Intensity. Independent of exercise variation being used, this is an absolute measure of intensity of load. If expressed as a percentage, $\text{AI} = \frac{\text{Load : kg}}{\text{1RM : kg}}$. Otherwise, AI specifies the weight of a training load.
- `AV`: Absolute Volume. Generally this is an expression of total work performed. For instance, if a trainee performs a total of 35 repetitions with a load of 205 kilograms, the absolute volume is $35 \cdot 205 = 7175 \text{kg}$.
- `INOL`: Intensity \* Number Of Lifts. Popularized by Hristo Hristov, this metric is commonly used to gauge the systemic stress induced by a training prescription, as well as to compare the relative stimulus incurred from multiple *groups* of sets with varying loading schemes and of varying quantities of repetitions. The name of this metric is a misnomer, as demonstrated by the non-linear denominator in the formula: $\text{INOL} = \frac{\text{Sets Performed} \cdot \text{Repetitions Per Set}}{100 - \text{(% Relative Intensity)} \cdot 100}$.
- `Intensity`: We use the term Intensity to refer to loads via various metrics. The most common unit is a percentage of one-repetition maximum load used, but may also be given as a RIR or RPE rating.
- `IA`: Intensity Amplitude. In performing similar or accessory exercises to a competition movement, a trainee will often be able to perform more or fewer repetitions of the variation than he would of the competition movement. Therefore IA is used to calculate RI, as it gives an indication of various factors like systemic stress (transitively via INOL). $\text{IA} = \frac{\text{Competition Movement 1RM}}{\text{Variation Movement 1RM}}$.
- `IM`: Intensity Modifier. Used for deloads or progressive overloads, often varying per microcycle, IM is a way of linearly scaling training loads for a set of training prescriptions. 
- `Maximum Repetitions`: The quantity of repetitions of a movement a trainee may perform in order to reach failure.
- `ORM` / `1RM`: One-Repetition Maximum. The maximum load that may be used such that a set of one repetition has a RIR rating of zero.
- `RI`: Relative Intensity. $\text{RI} = \frac{\text{AI} \cdot \text{IM}}{\text{IA}}$.
- `REQ`: Repetition-Endurance Quotient. This is used as a linear gauge of how close to failure a set is. The formula is $\text{REQ} = \frac{\text{Repetitions Performed}}{\text{Maximum Repetitions}}$. This metric may be used to predict technical quality of repetitions in a set.
- `R-INOL`: Relative INOL. This is a somewhat well-known metric for scaling INOL ratings across different types of movements, so that their systemic stress can be compared and used to evaluate overall training stimulus. I argue that this system is too subjective and imprecise, varying excessively in precision from one trainee to another. A more empirical procedural method is used to derive overall stimulus needed to achieve a desired adaptive effect.
- `RIR`: Repetitions In Reserve. After completing a set, RIR denotes the quantity of *additional* repetitions that would be required to reach failure. For instance, if one is able to complete 10 repetitions, but only performs 6, the set has a RIR rating of 4.
- `RIR falloff`: Given a training prescription with at least the following constraints: `_xB @ L | XX:XX` (quantity of sets is not required), RIR falloff is an integer-valued function (typically linear) with real-valued result ($\text{RIR falloff} : \mathbb{N} \mapsto \mathbb{R}$) expressing how much RIR will decrease (or, perhaps usefully in the context of circuit training, increase) per set performed.
- `RV`: Relative Volume. This is generally an expression of work performed in multiples of a 1RM load, given by the equation $\text{RV} = \text{RI} \cdot \text{Total Repetitions Performed}$. For instance, if a trainee performs a total of 35 repetitions at 82.5% of 1RM, this may be expressed as a relative volume of $35 \cdot 0.825 = 28.875$.
- `SV`: Set Volume. Simply put, this metric denotes sets total sets performed of some group of movements over a certain period of time.

In [0]:
from typing import Callable, List, Tuple, Union

In [0]:
import numpy as np
import pandas as pd

columns = ["Movement", "Sets", "Repetitions", "Load (% 1RM)"]

def add_row(df: pd.DataFrame, row: List[Union[str, int, float]]) -> None:
  return df.append(dict(zip(columns, row)), ignore_index=True)

df = pd.DataFrame(columns=columns)

rows = [
  ["Squat", 3, 5, 0.8],
  ["Bench Press", 1, 15, 0.65],
  ["Bench Press", 1, 12, 0.675],
  ["Bench Press", 3, 10, 0.7],
  ["Front Squat", 4, 4, 0.875],
]

for row in rows:
  global df
  df = add_row(df, row)

df["INOL"] = df["Sets"] * df["Repetitions"] / (1 - df[r"Load (% 1RM)"]) / 100
df

,Movement,Sets,Repetitions,Load (% 1RM),INOL
0,Squat,3,5,0.800,0.75
1,Bench Press,1,15,0.650,0.428571
2,Bench Press,1,12,0.675,0.369231
3,Bench Press,3,10,0.700,1
4,Front Squat,4,4,0.875,1.28


In [0]:
df_mvmt = df.groupby("Movement")

In [0]:
print("Total INOL")

df_mvmt.apply(lambda mvmt: sum(mvmt["INOL"]))

Total INOL


Movement
Bench Press    1.797802
Front Squat    1.280000
Squat          0.750000
dtype: float64

In [0]:
print("Set Volume")

df_mvmt.apply(lambda mvmt: sum(mvmt["Sets"]))

Set Volume


Movement
Bench Press    5
Front Squat    4
Squat          3
dtype: int64

In [0]:
print("Total Repetitions")

df_mvmt.apply(lambda mvmt: np.dot(mvmt["Sets"], mvmt["Repetitions"]))

Total Repetitions


Movement
Bench Press    57
Front Squat    16
Squat          15
dtype: int64

# One-Rep Maximum Formulas

In the following equations, $r$ denotes maximum number of repetitions possible and $w$ denotes weight of training load.

### Epley

$\text{1RM} = w \left( 1 + \frac{r}{30} \right)$

### Brzycki

$\text{1RM} = w \cdot \frac{36}{37 - r}$

### McGlothin

$\text{1RM} = \frac{100w}{101.3 - 2.67123r}$

### Lombardi

$\text{1RM} = w\sqrt[10]{r} = wr^{0.1}$

### Mayhew

$\text{1RM} = \frac{100w}{52.2 + 41.9e^{-0.055r}}$

### O'Conner

$\text{1RM} = w \left( 1 + \frac{r}{40} \right)$

### Wathen

$\text{1RM} = \frac{100w}{48.8 + 53.8e^{-0.075r}}$